# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [22]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [23]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser


### Config

In [24]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [25]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""
sys_prompt_4 = """Embed in the body of the code block the instructions you created prepended with a '\n'."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)
sys_prompt_list.append(sys_prompt_4)

sys_prompt = "".join(sys_prompt_list)

Generate the message and make the call

In [26]:
prompts = f"""
system: {sys_prompt}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

parser = MessageParser(prompts)
messages = parser.parse_messages()

# Get the response
# I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


Get a parser object, using a custom class.

In [27]:
parser = ResponseParser(response=response)

get a pandas dataframe from the parser

In [28]:
log_df = parser.get_log_df()

In [29]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.

In [30]:
parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-73v9FHH2bnrxZOT4rIixo3uSWRB4I,1681169017.000000,gpt-3.5-turbo-0301,82.000000,"# storing fibonacci sequence up to nth numberdef fibonacci(n: int) -> list: if n == 0: return [] if n == 1: return [0] seq = [0, 1] while len(seq) < n: seq.append(seq[-1] + seq[-2]) return seq# getting random number count and printing the fibonacci sequenceimport randomn = random.randint(1, 20)seq = fibonacci(n)print(f""Fibonacci sequence up to {n}:"")print(seq)def zero_pad(s: str) -> str: p = 10 - len(s) return '0'*p + s# Testing with a sample strings = ""hello""padded_s = zero_pad(s)print(f""Before Padding: '{s}'\nAfter Padding: '{padded_s}'"")"
